# FormulaFS

Run a formula, handle side effects, output buffering, etc. Serve the output via FS interface.

In [145]:
from fs.base import FS
from fs.info import Info
from fs.enums import ResourceType
from fs.errors import DirectoryExpected
from io import BytesIO

class FormulaFS(FS):
    """
    Serve the output of a command via FS API.
    
    Handle buffering.
    """
    def __init__(self, formula_dict):
        super().__init__()
        if(type(formula_dict) == dict):
            self._formula_dict = formula_dict
            
            if 'fn' in formula_dict:
                self._formula_name = formula_dict['fn']
            else:
                raise Exception("FormulaFS requires a function 'fn' in the initialization dict.")
            self._buffer = None
        else:
            raise TypeError('FormulaFS requires a dict to initialize.')
        
    def getinfo(self, path, namespaces=['basic']):
        nodetype = ResourceType.unknown
        return Info({
            'basic': {
                 'name':self._formula_name,
                 'is_dir': False
             },
             'details': {
                 'type': 0
             }
        })
    
    def listdir(self, path):
        raise DirectoryExpected(path)
    
    def makedir(self, path, *args, **kwargs):
        pass
    
    def openbin(self, path, *args, **kwargs):
        buf = self._formula_dict['fn'](*(self._formula_dict['args']))
        return BytesIO(buf.encode())
    
    def remove(self, path, *args, **kwargs):
        pass
    
    def removedir(self, path, *args, **kwargs):
        pass
    
    def setinfo(self, path, *args, **kwargs):
        pass
    

In [146]:
# Setup
def hello(msg='Default hello.'):
    print(msg)
    return "Return hello."

In [147]:
# Test: __init__()
my_formula = FormulaFS({'fn':hello, 'args':['Arg hello.']})

In [148]:
# Test
my_formula.isdir('/')

False

In [149]:
# Test
info = my_formula.getinfo('')
print(info)
print(info.name)

<file '<function hello at 0x7f5814082dd0>'>
<function hello at 0x7f5814082dd0>


In [150]:
# Test
res = my_formula.gettext('')

Arg hello.


In [151]:
print(res)

Return hello.
